In [1]:
from sklearn.preprocessing import KBinsDiscretizer
import numpy as np
import time
import math
import gym
from typing import Tuple

In [2]:
env = gym.make('CartPole-v1', render_mode="human")

In [3]:
# Configuración para discretizar estados
n_bins = (6, 12)  # Número de divisiones para discretizar
lower_bounds = [env.observation_space.low[2], -math.radians(50)]
upper_bounds = [env.observation_space.high[2], math.radians(50)]

# Discretizador de estados
def discretizer(_, __, angle, pole_velocity) -> Tuple[int, ...]:
    """Convierte un estado continuo a un estado discreto."""
    est = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='uniform')
    est.fit([lower_bounds, upper_bounds])
    return tuple(map(int, est.transform([[angle, pole_velocity]])[0]))


In [4]:
# Inicialización de la tabla Q
Q_table = np.zeros(n_bins + (env.action_space.n,))

# Política basada en la tabla Q
def policy(state: tuple):
    """Elige una acción basada en la política epsilon-greedy."""
    return np.argmax(Q_table[state])

# Actualización del valor Q usando aprendizaje por diferencias temporales
def new_Q_value(reward: float, new_state: tuple, discount_factor=1.0) -> float:
    """Calcula el nuevo valor Q."""
    future_optimal_value = np.max(Q_table[new_state])
    learned_value = reward + discount_factor * future_optimal_value
    return learned_value

# Tasa de aprendizaje adaptativa
def learning_rate(n: int, min_rate=0.01) -> float:
    """Calcula una tasa de aprendizaje que decrece con el tiempo."""
    return max(min_rate, min(1.0, 1.0 - math.log10((n + 1) / 25)))

# Tasa de exploración adaptativa
def exploration_rate(n: int, min_rate=0.1) -> float:
    """Calcula una tasa de exploración que decrece con el tiempo."""
    return max(min_rate, min(1, 1.0 - math.log10((n + 1) / 25)))


In [ ]:
# Parámetros del entrenamiento
n_episodes = 10000
discount_factor = 0.99  # Factor de descuento para el valor futuro

for e in range(n_episodes):
    obs, _ = env.reset()
    current_state, done = discretizer(*obs), False

    while not done:
        # Seleccionar acción según la política epsilon-greedy
        if np.random.random() < exploration_rate(e):
            action = env.action_space.sample()  # Explorar
        else:
            action = policy(current_state)  # Explotar

        # Ejecutar la acción en el entorno
        obs, reward, done, _, _ = env.step(action)
        new_state = discretizer(*obs)

        # Actualizar la tabla Q
        lr = learning_rate(e)
        learnt_value = new_Q_value(reward, new_state, discount_factor)
        old_value = Q_table[current_state][action]
        Q_table[current_state][action] = (1 - lr) * old_value + lr * learnt_value

        current_state = new_state

        # Renderizar el entorno (opcional)
        if e % 100 == 0:  # Renderizar solo en episodios seleccionados
            env.render()
env.close()


C:\Python312\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
